In [2404]:
#Customers who left are -1, current customers are 1
#Males are 0, females are 1


#binoy says:
#try out imbalanced random forest FROM IMBLEARN

#add github project overview


In [2405]:
import pandas as pd
import numpy as np
from pathlib import Path


from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [2406]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners2.csv"))

starter_df = starter_df[starter_df != 'Unknown'].dropna()
starter_df.sample(5)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
4851,780201633,Attrited Customer,41,F,3,Graduate,Single,$40K - $60K,Blue,31,...,5655.0,1154,4501.0,0.702,2189,51,0.700,0.204,0.994670,0.00533
7285,811942983,Existing Customer,55,M,2,High School,Married,$120K +,Blue,50,...,17627.0,1756,15871.0,0.757,4836,74,0.721,0.100,0.000186,0.99981
2486,714030183,Existing Customer,59,M,0,Graduate,Single,$80K - $120K,Silver,36,...,34516.0,1494,33022.0,1.229,3259,65,0.585,0.043,0.000235,0.99977
4678,711406758,Existing Customer,48,F,3,Graduate,Married,Less than $40K,Blue,36,...,1667.0,0,1667.0,0.686,4385,80,0.951,0.000,0.000118,0.99988
9169,794962983,Existing Customer,47,F,3,Graduate,Divorced,Less than $40K,Blue,41,...,9947.0,2496,7451.0,0.802,14576,110,0.774,0.251,0.000096,0.99990


In [2407]:
#Change a few column names so they display better in the early DF
replacement_column_names = {'Attrition_Flag': 'Status',
        'Customer_Age': 'Age',
        'Education_Level': 'Education',
        'Dependent_count': 'Dependents',
        'Income_Category': 'Income',
        'Card_Category': 'Card Type',
        'Months_on_book': 'Tenure(month)'
        }
starter_df.rename(columns=replacement_column_names, inplace=True)

In [2408]:
#Remove columns that are not understood or irrelevant 
#The Naive Bayes Classifiers give a perfect score, it could be linked to our target(y)
starter_df.drop(columns=[
#'Total_Relationship_Count', #Going to try adding this and StandardScaling
#'Contacts_Count_12_mon', #Going to try adding this and StandardScaling 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', #Going to try adding this and StandardScaling 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', #Going to try adding this and StandardScaling 
#'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1', #Going to try adding these and StandardScaling 
'CLIENTNUM' #unnecessary
],
inplace=True
)


In [2409]:
"""
The education category 'Graduate' is vague/ambiguous 
In 2021, the highest level of education of the population age 25 and older in the United States was distributed as follows: 
    8.9% had less than a high school diploma or equivalent.
        our dataset shows 17% uneducated
    27.9% had high school graduate as their highest level of school completed. 
        our dataset shows 23.3%
    14.9% had completed some college but not a degree.
        ours only has an ambiguous 'graduate' as 36.6%. could be this, could mean post-grad, term is used interchangeably.
    10.5% had an associate degree as their highest level of school completed.
        our dataset has no distinction between associate/undergrad. just says 'college'. ours shows 11.9% for college
    23.5% had a bachelors degree as their highest degree.
        ours shows 11.9% for college
    14.4% had completed an advanced degree such as a masters degree, professional degree or doctoral degree. 
        ours shows 6.1% post-graduate, 5.1% doctorate, and the ambiguous 36.6% 'graduate'ArithmeticError

#Below is a quick dataframe to show the value counts and percentages for the Education column
#This should help evaluate the impact of the Graduate column considering its ambiguous classification
counts = starter_df['Education'].value_counts()
percs = starter_df['Education'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])


It's hard to tell what they mean by 'graduate'. Unfortunately, its the bulk of our data, and dropping it will hurt our sample size. 
    The existence of a 'college' column suggests that graduate could mean beyond college. 
        Alternatively, it could mean you attended college but did not graduate, and 'graduate' represents standard undergrads.
Its a high amount of the data (36.6%), so its likely a mix of multiple categories not included, i.e. trade schools, associate degrees, dropouts, specializations
With so little info on what Graduate means, I'm dropping it all. 
We could try to create a weighting system that roughly ranks the other columns based on years at school (?)
    Essentially:
        uneducated = 0, highschool=1, college=2, post-grad=3, doctorate=4

"""
#OPTION1
#Change education to ranking system, drop graduate
#starter_df = starter_df[starter_df.Education != 'Graduate']
#starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 1, 'College' : 2, 'Post-Graduate' : 3, 'Doctorate' : 4}})
#starter_df.Education = starter_df.Education.astype(int)

#OPTION2
#Here is another option which includes the Graduate data.
starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 1, 'College' : 2, 'Graduate' :  3, 'Post-Graduate' : 4, 'Doctorate' : 5}})
starter_df.Education = starter_df.Education.astype(int)



In [2410]:
#Replace card categories from Blue/Silver/Gold/Plat to 0/1/2/3/4
starter_df = starter_df.replace({'Card Type' : {'Blue': 0, 'Silver': 1, 'Gold' : 3, 'Platinum' : 4}})
starter_df.loc[ : ,['Card Type']] = starter_df.loc[ : ,['Card Type']].astype(int)

#Replace M/F with 0,1
starter_df = starter_df.replace({'Gender' : {'M': 0, 'F': 1}})
starter_df.Gender = starter_df.Gender.astype(int)
#NOT BEING USED CURRENTLY, Replaces M/F with -1/1
#starter_df = starter_df.replace({'Gender' : {'M': '-1', 'F': '1'}})

#Replace customer's status to -1 if they left and 1 if they stayed
#starter_df = starter_df.replace({'Status' : {'Attrited Customer': -1, 'Existing Customer': 1}})
#starter_df.Status = starter_df.Status.astype(int)

#Replace customer's status to 0 if they left and 1 if they stayed
#Another feature option not being used
starter_df = starter_df.replace({'Status' : {'Attrited Customer': 0, 'Existing Customer': 1}})



In [2411]:
#binoy says dont use this

"""
use feature creation to make a ratio between dependents and income sources
consider someone divorced as Single. only 10% of divorced people receive alimony payments
        either a 1 (single/divorced) or a 2 (married)
        divide that by the amount of dependents plus one, which represents how many people you take care of
                i.e. a single person with no dependents only takes care of themself, so they are a 1. 
                """

starter_df = starter_df.replace({'Marital_Status' : {'Divorced': 1, 'Single': 1, 'Married' : 2}})
starter_df['MaritalDependentRatio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)




In [2412]:
'''
commented out because I'm trying out the forbidden feature I created in the cell above 
#OneHotEncode the categorical variable 'Marital Status'
enc = OneHotEncoder(sparse=False)
categorical_features = ["Marital_Status"]
encoded_data = enc.fit_transform(starter_df[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_features))

#Add the encoded data DF to the original DF
starter_df = pd.concat([encoded_df, starter_df], axis=1)
starter_df = starter_df.dropna()
starter_df.drop(columns='Marital_Status', inplace=True)
starter_df.Marital_Status_Divorced, starter_df.Marital_Status_Married, starter_df.Marital_Status_Single = starter_df.Marital_Status_Divorced.astype(int), starter_df.Marital_Status_Married.astype(int), starter_df.Marital_Status_Single.astype(int)
'''

'\ncommented out because I\'m trying out the forbidden feature I created in the cell above \n#OneHotEncode the categorical variable \'Marital Status\'\nenc = OneHotEncoder(sparse=False)\ncategorical_features = ["Marital_Status"]\nencoded_data = enc.fit_transform(starter_df[categorical_features])\nencoded_df = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_features))\n\n#Add the encoded data DF to the original DF\nstarter_df = pd.concat([encoded_df, starter_df], axis=1)\nstarter_df = starter_df.dropna()\nstarter_df.drop(columns=\'Marital_Status\', inplace=True)\nstarter_df.Marital_Status_Divorced, starter_df.Marital_Status_Married, starter_df.Marital_Status_Single = starter_df.Marital_Status_Divorced.astype(int), starter_df.Marital_Status_Married.astype(int), starter_df.Marital_Status_Single.astype(int)\n'

In [2413]:
#Do feature creation to make a ratio between avg open to buy and credit limit
starter_df['CreditUsage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)
starter_df = starter_df.dropna()

In [2414]:
#Do feature creation to make a ratio between transaction amount and trans count
starter_df['AvgTransValue'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

In [2415]:
#Do feature creation to make a ratio between age and tenure
starter_df['TenureByAge'] = (starter_df['Tenure(month)'] / starter_df['Age']).round(2)
starter_df.drop(columns=['Tenure(month)', 'Age'], inplace=True)

In [2416]:
#Rank the income as a 0-4 low-high
starter_df = starter_df.replace({'Income' : {'Less than $40K': 0, '$40K - $60K': 1, '$80K - $120K' : 2, '$60K - $80K' : 3, '$120K +' : 4}})
starter_df.Income = starter_df.Income.astype(int)


In [2417]:
starter_df.sample(5)

,Status,Gender,Education,Income,Card Type,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,MaritalDependentRatio,CreditUsage,AvgTransValue,TenureByAge
6464,0,1,3,1,0,5,3,4,1651,0.790,0.560,0.279,0.10,0.72,61.64,0.78
9630,1,0,1,4,0,3,1,2,939,0.826,0.754,0.027,0.33,0.97,145.35,1.00
3642,1,0,0,3,0,4,1,3,1942,0.926,0.853,0.558,0.20,0.44,55.75,0.86
6377,1,0,3,1,0,5,3,2,0,0.885,1.034,0.000,0.20,1.00,76.15,0.45
1393,1,0,3,3,0,3,1,4,0,0.877,1.105,0.000,0.33,1.00,42.42,0.69


In [2418]:
#Define features(X) and target(y)
y = starter_df['Status']
X = starter_df.drop(columns=['Status', 
#'Education', 'TenureByAge', 'Gender', 'Dependents',
#'CreditUsage', 'AvgTransValue', 'Card Type' 
])
#Going to try and remove all things related to Edu/age/dependents, see what happens

In [2419]:
#Split features and target in to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [2420]:
#Scale all numerical/float values that don't represent categories

scaler = StandardScaler()

#Use Column Transformer to scale only the numerical/float values that don't represent categories (male/female, married/single/divorced)
#Going to try and remove all things related to Edu/age/dependents, see what happens
col_tran= ColumnTransformer([
('Total_Revolving_Bal_Scaled', scaler, ['Total_Revolving_Bal']),
#('Dependents_Scaled', scaler, ['Dependents']),
('Education_Scaled', scaler, ['Education']),
('Income_Scaled', scaler, ['Income']),
('Card Type Scaled', scaler, ['Card Type']),
('Months_Inactive_12_mon_scaled', scaler, ['Months_Inactive_12_mon']),
('CreditUsageScaled', scaler, ['CreditUsage']),
('AvgTransValueScaled', scaler, ['AvgTransValue']),
('TenureByAgeScaled', scaler, ['TenureByAge']),
('Avg_Util_Ratio_Scaled', scaler, ['Avg_Utilization_Ratio']),
('Total_Relationship_Count_Scaled', scaler, ['Total_Relationship_Count']),
('Contacts_Count_12_mon_scaled', scaler, ['Contacts_Count_12_mon']),
('Total_Amt_Chng_Q4_Q1_scaled', scaler, ['Total_Amt_Chng_Q4_Q1']),
('Total_Ct_Chng_Q4_Q1_scaled', scaler, ['Total_Ct_Chng_Q4_Q1']),
('MaritalDependentRatioScaled', scaler, ['MaritalDependentRatio'])
])


X_train = col_tran.fit_transform(X_train)
X_test = col_tran.transform(X_test)


In [2421]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [2422]:
#Choose a model

#GBC STILL THE BEST
#TRY WITH BOTH EDUCATION COLUMN OPTIONS (with and without Graduate data)

clf = GradientBoostingClassifier(
n_estimators=500,    #default = 100    range = 1-inf
random_state= 2,     #default = None   range = 1-inf
#subsample= 1,     #default = 1   range = 0.-1
#min_samples_split = 2,      #default = 2   range = 2-inf
#max_depth=3,  #default = 3    range = 1-inf
#min_impurity_decrease=0,    #default = 0    range = 0 - inf
#min_samples_leaf = 1,            #default = 1   range = 1 - inf
#min_weight_fraction_leaf = 0,     #default = 0   range =0 - 0.5
#max_leaf_nodes = None,     #default = None   range = 2-inf
#learning_rate = 0.1          #default=0.1    range 0.0-inf
)


'''
clf = XGBClassifier(
n_estimators = 50, 
max_depth = 4, 
objective='binary:logistic'
)
'''

#clf = ExtraTreesClassifier()

#clf = AdaBoostClassifier()
#clf = AdaBoostClassifier(n_estimators=200, random_state=2, learning_rate = 0.2)

#svc = SVC()
#clf = AdaBoostClassifier(base_estimator=svc, algorithm='SAMME')

#clf = BalancedRandomForestClassifier()

#clf = SVC()

#clf = RandomForestClassifier()

"\nclf = XGBClassifier(\nn_estimators = 50, \nmax_depth = 4, \nobjective='binary:logistic'\n)\n"

In [2423]:
#Fit the model on the training data
clf.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=500, random_state=2)

In [2424]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [2425]:
# Evaluate the model's performance
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 92.49%


In [2426]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[ 203   74]
 [  59 1435]]


In [2427]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.77      0.73      0.75       277
           1       0.95      0.96      0.96      1494

    accuracy                           0.92      1771
   macro avg       0.86      0.85      0.85      1771
weighted avg       0.92      0.92      0.92      1771



In [2428]:
'''
*current best
clf = GradientBoostingClassifier(
n_estimators=500,    #default = 100    range =
random_state=2,     #default = None   range =
subsample=1,     #default = 1   range =
min_samples_split = 4,      #default = 2   range = 2-inf
max_depth=3,  #default = 3    range = 1-inf
min_impurity_decrease=0,    #default = 0    range = 0 - inf
min_samples_leaf = 1,            #default = 1   range = 1 - inf
min_weight_fraction_leaf = 0,     #default = 0   range =0 - 0.5
max_leaf_nodes = None     #default = None   range = 2-inf
)
gives = 76/73/72 acc=92
#similar scoress when adding max_features=10 and keeping the rest above the same


'''

'\n*current best\nclf = GradientBoostingClassifier(\nn_estimators=500,    #default = 100    range =\nrandom_state=2,     #default = None   range =\nsubsample=1,     #default = 1   range =\nmin_samples_split = 4,      #default = 2   range = 2-inf\nmax_depth=3,  #default = 3    range = 1-inf\nmin_impurity_decrease=0,    #default = 0    range = 0 - inf\nmin_samples_leaf = 1,            #default = 1   range = 1 - inf\nmin_weight_fraction_leaf = 0,     #default = 0   range =0 - 0.5\nmax_leaf_nodes = None     #default = None   range = 2-inf\n)\ngives = 76/73/72 acc=92\n#similar scoress when adding max_features=10 and keeping the rest above the same\n\n\n'